In [51]:
import IPython.display as ipd
import numpy as np
from keras.layers import Conv1D, Dense, Activation, Input, Reshape, Flatten, MaxPooling1D, BatchNormalization
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
import sys, random

In [52]:
sys.path.append("data/")

In [3]:
from load_data import load_GAN_Y, load_Discr_Y

In [129]:
gan_X = np.load("data/audio1_gan_X.npy")
gan_X = gan_X.reshape([gan_X.shape[0], gan_X.shape[1], 1])
discr_X_true = np.load("data/audio1_disc_X.npy")

In [174]:
def create_generator(nodes):
    model = Sequential()
    
    model.add(Conv1D(nodes,(20,), strides=1, input_shape=(66150,1), padding='same'))
    model.add(LeakyReLU())
    
    model.add(Conv1D(nodes,(10,), strides=1, input_shape=(66150,1), padding='same'))
    model.add(LeakyReLU())
    
    model.add(Conv1D(1, (3,), padding='same', activation='tanh'))
    model.add(Reshape([66150,]))

    
    
    return model

"""
def create_discriminator(nodes):
    model = Sequential()
    
    model.add(Reshape([66150,1], input_shape=(66150,)))
    
    model.add(Conv1D(nodes,(20,), strides=8))
    model.add(LeakyReLU())
    model.add(MaxPooling1D())
    
    model.add(Conv1D(nodes,(12,), strides=4))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    
    model.add(Flatten())
    
    model.add(Dense(nodes//2))
    model.add(LeakyReLU())
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['accuracy'])
    
    return model
"""


def create_discriminator(nodes):
    model = Sequential()
    
    model.add(Dense(nodes, input_shape=(66150,), activation='relu'))
    
    model.add(Dense(nodes, activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

def create_models(gen_nodes=128, discr_nodes=128):
    
    gen = create_generator(gen_nodes)
    discr = create_discriminator(discr_nodes)

    sound = Input(shape=(66150,1,))
    out_gen = gen(sound)
    pred = discr(out_gen)
    
    model = Model(sound, pred)
    
    model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    return gen, discr, model
    

In [173]:
gen.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_141 (Conv1D)          (None, 66150, 128)        768       
_________________________________________________________________
leaky_re_lu_119 (LeakyReLU)  (None, 66150, 128)        0         
_________________________________________________________________
conv1d_142 (Conv1D)          (None, 66150, 1)          385       
_________________________________________________________________
reshape_44 (Reshape)         (None, 66150)             0         
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


In [175]:
gen, discr, model = create_models()

In [165]:
def shuffle(X, Y):
    np.arange(X.shape[0])
    
    

In [166]:
def train(GAN, Discr, model, GAN_X, Discr_X_true, epochs=1):
    
    length = GAN_X.shape[0]
    
    Discr_Y_true = load_Discr_Y(length, False)
    Discr_Y_false = load_Discr_Y(length, True)
    Discr_Y = np.append(Discr_Y_true, Discr_Y_false).reshape([length*2, 1])

    GAN_Y = load_GAN_Y(length)
    
    train_discr = True
    train_gen = True
    
    
    for i in range(epochs):
    
        
        if train_discr:
            
            Discr.trainable=True

            Discr.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

            print("Training Discriminator")
            # build discriminator training set
            Discr_X_false = GAN.predict(GAN_X)

            Discr_X = np.append(Discr_X_true, Discr_X_false, axis=0)

            output = discr.fit(Discr_X, Discr_Y, shuffle="true")
            
            print(output.history["acc"])
            if output.history["acc"][0] < .6:
                train_gen = False
            else:
                train_gen = True
        
        if train_gen:
        
        
            Discr.trainable=False

            model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])


            print("Training Generator")

            output = model.fit(GAN_X, GAN_Y)
            
            if output.history["acc"][0] < .8:
                train_discr = False
            else:
                train_discr = True

In [167]:
train(gen, discr, model, gan_X, discr_X_true, 7)

Training Discriminator
Epoch 1/1
330/330 [==============================] - 4s 13ms/step - loss: 0.7048 - acc: 0.7848
[0.7848484852097252]
Training Generator
Epoch 1/1
165/165 [==============================] - 30s 181ms/step - loss: 1.1098 - acc: 0.5030
Training Generator
Epoch 1/1
165/165 [==============================] - 30s 179ms/step - loss: 0.9670 - acc: 0.6242
Training Generator
Epoch 1/1
165/165 [==============================] - 30s 180ms/step - loss: 0.9400 - acc: 0.6303
Training Generator
Epoch 1/1
165/165 [==============================] - 30s 180ms/step - loss: 0.9172 - acc: 0.5939
Training Generator
Epoch 1/1
165/165 [==============================] - 30s 181ms/step - loss: 0.9182 - acc: 0.6182
Training Generator
Epoch 1/1
165/165 [==============================] - 30s 180ms/step - loss: 0.9185 - acc: 0.6000
Training Generator
Epoch 1/1
165/165 [==============================] - 30s 180ms/step - loss: 0.9553 - acc: 0.6000


In [168]:
model.evaluate(gan_X, load_GAN_Y(gan_X.shape[0]))

165/165 [==============================] - 10s 63ms/step


[16.11809539794922, 0.0]

In [172]:
x=133
ipd.Audio(gen.predict(gan_X[x:x+1]), rate=22050)

In [69]:
discr.predict(gen.predict(gan_X[0:1]))

array([[1.]], dtype=float32)